In [52]:
# Libraries
%load_ext autoreload
%autoreload

import numpy as np
import iris
import iris.coord_categorisation
import pandas as pd

import sys
sys.path.append('../')
from libs.plot_maps import *


import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy.ma as ma
%matplotlib inline
import cartopy.crs as ccrs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading in the data (ignoring precipation because (a) we don't need it and (b) it has additional dimension coordinates (lat_bnds and lon_bnds) which upset iris.

In [64]:
dir = '../data/climate/'
files      = { 'air'               : 'air.mon.mean.nc',
               'lightning'         : 'lightn_climatology_otd_mlnha.nc',
               'relative_humidity' : 'rhum.mon.mean.nc'}

input_data = {}

for key, f in files.items():
    print(key)
    dat = iris.load_cube(dir + f)
    input_data[key] = dat

air
lightning
relative_humidity


A for loop to save the arrays of the original coordinates, remove them and then re-add them to the data set. Similar to the regridding script, we're going to use the masked climate file to pinch the coordinates array.

In [65]:
mask = iris.load_cube(dir + 'climate_mask.nc')

m_lat = mask.coord('latitude').points
m_lon = mask.coord('longitude').points

In [66]:
# print(input_data['lightning'])
# print(input_data['air'])
# print(input_data['relative_humidity'])
# input_data['air'].collapsed('time', iris.analysis.MEAN).coord('latitude')

In [67]:
for key in input_data.keys():

    if key == 'air' or 'relative_humidity':

        # Adding back in the lat coord
        coord_lat = iris.coords.DimCoord(m_lat,standard_name='latitude',
                                         coord_system=iris.coord_systems.GeogCS(6371229.0),var_name='latitude',
                                         units='degrees')
        input_data[key].add_dim_coord(coord_lat, 1)

        # Adding back in the lon coord
        coord_lon = iris.coords.DimCoord(m_lon, standard_name='longitude',
                                         coord_system=iris.coord_systems.GeogCS(6371229.0),var_name='longitude',
                                         units='degrees')
        input_data[key].add_dim_coord(coord_lon, 2)

        # Also adding a year
        iris.coord_categorisation.add_year(input_data[key], 'time', name='year')

ValueError: A dim_coord is already associated with dimension 1.

In [68]:
print(input_data['air'].coord('time'))
# iris.coord_categorisation.add_year(input_data['air'], 'time', name='year')

DimCoord([1948-01-01 00:00:00, 1948-02-01 00:00:00, 1948-03-01 00:00:00,
       1948-04-01 00:00:00, 1948-05-01 00:00:00, 1948-06-01 00:00:00,
       1948-07-01 00:00:00, 1948-08-01 00:00:00, 1948-09-01 00:00:00,
       1948-10-01 00:00:00, 1948-11-01 00:00:00, 1948-12-01 00:00:00,
       1949-01-01 00:00:00, 1949-02-01 00:00:00, 1949-03-01 00:00:00,
       1949-04-01 00:00:00, 1949-05-01 00:00:00, 1949-06-01 00:00:00,
       1949-07-01 00:00:00, 1949-08-01 00:00:00, 1949-09-01 00:00:00,
       1949-10-01 00:00:00, 1949-11-01 00:00:00, 1949-12-01 00:00:00,
       1950-01-01 00:00:00, 1950-02-01 00:00:00, 1950-03-01 00:00:00,
       1950-04-01 00:00:00, 1950-05-01 00:00:00, 1950-06-01 00:00:00,
       1950-07-01 00:00:00, 1950-08-01 00:00:00, 1950-09-01 00:00:00,
       1950-10-01 00:00:00, 1950-11-01 00:00:00, 1950-12-01 00:00:00,
       1951-01-01 00:00:00, 1951-02-01 00:00:00, 1951-03-01 00:00:00,
       1951-04-01 00:00:00, 1951-05-01 00:00:00, 1951-06-01 00:00:00,
       1951-07-01

In [59]:
lightning = input_data['air'].copy()



lightn / (1)                        (time: 12; latitude: 360; longitude: 720)
     Dimension coordinates:
          time                           x             -               -
          latitude                       -             x               -
          longitude                      -             -               x
     Attributes:
          comment_1: LIS/OTD 0.5 Degree High Resolution Monthly Climatology (HRMC)
          comment_2:: non-transient
          comment_3:: (i.e, average lightening in period of oberservation for each month)
          comment_4:: Units: Flash count per km2
          history: Wed Apr  2 19:04:33 2008: ncap -O -s lightn[time,lat,lon]=float(lightn*10000.)...
